# Generate all figures

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import hypothesis as h
from itertools import cycle

import time
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
matplotlib.rcParams['text.usetex'] = True

In [ ]:
from plotting_tools import make_dirs_safe
directory = 'figures/'
make_dirs_safe(directory)
lines = ["-", "--", "-.", ":"]
linecycler = cycle(lines)

In [ ]:
from trajectory_creator import get_trajectory
from plotting_tools import add_scalebar, remove_ticks

np.random.seed(0)

fig, axis = plt.subplots(1, 4, figsize=(5, 1.2), sharey='row',
                        gridspec_kw={'hspace': 0, 'wspace': 0.1})
traj = get_trajectory("clover")
traj.scale_bounding_box([5, 5])
error = np.random.normal(size=traj.coeffs.shape)
error = error/np.linalg.norm(error)
# todo scale the trajectory
for idx, scale in enumerate([0.1, 0.2, 0.4, 0.8]):
    ax = axis[idx]
    traj.plot(ax=ax, linestyle='--')
    perturbed = traj.copy()
    perturbed.coeffs += scale*error
    perturbed.plot(ax=ax)
    ax.axis('equal')
    ax.set_title("error: {}m".format(scale))
    add_scalebar(ax, loc='upper left', size=1)
    remove_ticks(ax)
# plt.tight_layout()
from plotting_tools import savefig
savefig(fig, directory+"error_overview.pdf")

In [ ]:
from plotting_tools import plot_noise
plt.rcParams['figure.figsize'] = 5, 2.5

In [ ]:
fig, axis = plt.subplots(2, 1, figsize=(5, 4), sharex='col', gridspec_kw={'hspace': 0.1, 'wspace': 0})

plot = plot_noise('noise_and_anchors',
                  error_types=['errors'],
                  save_figures=True,
                  min_noise=0,
                  max_noise=None,
                  background_alpha=0.2,
                  start=30,
                  anchors=True,
                  ax=axis[1])
plot = plot_noise('noise_right_inverse_weighted',
                  error_types=['errors'],
                  save_figures=False,
                  max_noise=None,
                  background_alpha=0.2,
                  start=100,
                  ax=axis[0])
plot.set_xticks([31, 62, 125, 250, 500])
axis[0].set_xlim(15, 500)
axis[1].set_ylim(1e-4, 1e-1)
axis[0].set_ylim(5e-4, 5e1)
axis[0].grid()
axis[1].grid()
axis[0].set_ylabel('errors')
axis[1].set_xlabel("number of measurements")
plt.tight_layout()
plt.savefig(directory + 'noise.pdf')

plt.show()

In [ ]:
plot = plot_noise('noise_to_square_right_inverse', error_types=['errors'], save_figures=False,  max_noise=None, background_alpha=0.2, start=100)
plot.set_xticks([15, 31, 62, 125, 250, 500])
plt.ylim(5e-4, 5e1)
plt.savefig(directory+'noise_to_quare_error.pdf')
plt.show()

In [ ]:
plot = plot_noise('noise_right_inverse', error_types=['errors'], save_figures=True,  max_noise=None, background_alpha=0.3, start=30)
plt.ylim(5e-4, 5e1)
plot.set_xticks([15, 31, 62, 125, 250, 500])
plt.savefig(directory+'noise_error.pdf')
plt.show()

In [ ]:
plot = plot_noise('noise_right_inverse_weighted', error_types=['errors'], save_figures=True,  max_noise=None, background_alpha=0.3, start=30)
plt.ylim(5e-4, 5e1)
plot.set_xticks([15, 31, 62, 125, 250, 500])
plt.savefig(directory+'noise_weighted_error.pdf')
plt.show()

In [ ]:
n_measurement_list = range(10, 40)

experiment_params={
    "n_dimensions": 2,
    "n_anchors": 4,
    "n_constraints": 5,
    "n_anchors_list": [3, 4, 5],
    "n_constraints_list": [3, 5, 7],
}

start = time.time()
probabilities_anchors = []
for idx, n_anchors in enumerate(experiment_params["n_anchors_list"]):
    print("{} anchors".format(n_anchors))
    probabilities_anchors.append([h.probability_upper_bound_any_measurements(
        n_dimensions=experiment_params["n_dimensions"],
        n_constraints=experiment_params["n_constraints"],
        n_measurements=n,
        position_wise=False,
        n_anchors=n_anchors,
        n_positions=np.Infinity,
        full_matrix=True
    ) for n in n_measurement_list])
probabilities_anchors = np.array(probabilities_anchors)
print("anchors total time: {:.2f}s".format(time.time()-start))

start = time.time()
probabilities_constraints = []
for idx, n_constraints in enumerate(experiment_params["n_constraints_list"]):
    print("{} constraints".format(n_constraints))
    probabilities_constraints.append([h.probability_upper_bound_any_measurements(
        n_dimensions=experiment_params["n_dimensions"],
        n_constraints=n_constraints,
        n_measurements=n,
        position_wise=False,
        n_anchors=experiment_params["n_anchors"],
        n_positions=np.Infinity,
        full_matrix=True,
    ) for n in n_measurement_list])
probabilities_constraints = np.array(probabilities_constraints)
print("constrains total time: {:.2f}s".format(time.time()-start))

In [ ]:
fig, axis = plt.subplots(2, 1, figsize=(5, 4), sharex='col', gridspec_kw={'hspace': 0.1, 'wspace': 0})

linecycler = cycle(lines)
for idx, n_anchors in enumerate(experiment_params["n_anchors_list"]):

    p = axis[0].step(
        n_measurement_list,
        probabilities_anchors[idx],
        label="{} anchors".format(n_anchors),
        where='post',
        linestyle=next(linecycler))
# ax.set_xlabel("number of measurements")
axis[0].set_ylabel("probability")
# axis[0].set_ylim(0)
axis[0].grid()
axis[0].legend(loc='lower right')

linecycler = cycle(lines)
for idx, n_constraints in enumerate(experiment_params["n_constraints_list"]):

    p = axis[1].step(
        n_measurement_list,
        probabilities_constraints[idx],
        label="K={}".format(n_constraints),
        where='post',
        linestyle=next(linecycler))
axis[1].set_xlabel("number of measurements")
axis[1].set_ylabel("probability")
# axis[1].set_ylim(0)
axis[1].grid()
axis[1].legend()
plt.tight_layout()
plt.savefig(directory + 'probability.pdf')

plt.show()

### An example how to make a break axis plot (not used in the paper)

In [ ]:
from matplotlib import gridspec

f, axis = plt.subplots(2, 1, sharex=True, figsize=(5, 1.5))
gs = gridspec.GridSpec(2, 1, height_ratios=[3.05, 1])
axis[0] = plt.subplot(gs[0])
axis[1] = plt.subplot(gs[1])

linecycler = cycle(lines)
for idx, n_anchors in enumerate(experiment_params["n_anchors_list"]):

    p = axis[0].step(
        n_measurement_list,
        probabilities_constraints[idx],
        label="{} anchors".format(n_anchors),
        where='post',
        linestyle=next(linecycler))
    p = axis[1].step(
        n_measurement_list,
        probabilities_constraints[idx],
        label="{} anchors".format(n_anchors),
        where='post',
        linestyle=p[0].get_linestyle())
# ax.set_xlabel("number of measurements")
# axis[0].set_ylabel("probability")
axis[0].set_ylim(0.7, 1.05)
axis[1].set_ylim(0, 0.1)
axis[0].grid()
axis[1].grid()
axis[1].legend(loc='lower right')


# hide the spines between ax and ax2
axis[0].spines['bottom'].set_visible(False)
axis[1].spines['top'].set_visible(False)
axis[0].xaxis.tick_top()
axis[0].tick_params(labeltop='off')  # don't put tick labels at the top
axis[1].xaxis.tick_bottom()

d = .01  # how big to make the diagonal lines in axes coordinates
# arguments to pass to plot, just so we don't keep repeating them
kwargs = dict(transform=axis[0].transAxes, color='k', clip_on=False)
axis[0].plot((-d, +d), (-d, +d), **kwargs)        # top-left diagonal
axis[0].plot((1 - d, 1 + d), (-d, +d), **kwargs)  # top-right diagonal

kwargs.update(transform=axis[1].transAxes)  # switch to the bottom axes
axis[1].plot((-d, +d), (1 - d, 1 + d), **kwargs)  # bottom-left diagonal
axis[1].plot((1 - d, 1 + d), (1 - d, 1 + d), **kwargs)  # bottom-right diagonal

plt.show()